In [1]:
import math
import yfinance as yf
import numpy as np
import pandas as pd
from datetime import date, timedelta, datetime
import time
import talib as ta

In [2]:
def get_intraday_stocklist():
  df = pd.read_csv('Intraday_stocks_list.csv')
  return df

In [3]:
def download_stock(tckr, start_date, end_date):
  start = start_date
  end = end_date
  symbol = tckr
  symbol = symbol+'.NS'
  stock_data = yf.download(symbol, start=start, end=end,rounding=True)
  if(len(stock_data)<1):
    stock_data = pd.DataFrame()
  return stock_data

In [4]:
def Gann_prediction(LTP):
  
  # find sq root of the ltp
  sq_rt_ltp = math.sqrt(LTP)
  #ceil the sq rt
  cieled_sq_rt = math.ceil(sq_rt_ltp)
  #get starting number for iteration
  lowest_num = cieled_sq_rt-2
  block = [lowest_num, lowest_num+1, lowest_num+2]
  
  for i in (block):
    centre_1 = math.pow(i+0.125, 2)
    centre_2 = math.pow(i+0.125*2, 2)
    centre_3 = math.pow(i+0.125*3, 2)
    centre_4 = math.pow(i+0.125*4, 2)
    centre_5 = math.pow(i+0.125*5, 2)
    centre_6 = math.pow(i+0.125*6, 2)
    centre_7 = math.pow(i+0.125*7, 2)
    centre_8 = math.pow(i+0.125*8, 2)
    if(i==block[0]):
      level_1 = [centre_1, centre_2, centre_3,centre_4,centre_5,centre_6,centre_7,centre_8]
    if(i==block[1]):
      level_2 = [centre_1, centre_2, centre_3,centre_4,centre_5,centre_6,centre_7,centre_8]
    if(i==block[2]):
      level_3 = [centre_1, centre_2, centre_3,centre_4,centre_5,centre_6,centre_7,centre_8]
  
  #merge list
  Gann_list = level_1 + level_2 + level_3


  insrt_loc = 0  
  for j in range(len(Gann_list)):
    if(Gann_list[j] < LTP < Gann_list[j+1]):
      insrt_loc = j+1
      Gann_list.insert(insrt_loc, LTP)
      break

  Resistance_1, Resistance_2, Resistance_3, Resistance_4 = (Gann_list[insrt_loc+2], Gann_list[insrt_loc+3],
                                                                       Gann_list[insrt_loc+4], Gann_list[insrt_loc+5])

  Support_1, Support_2, Support_3, Support_4 = (Gann_list[insrt_loc-2], Gann_list[insrt_loc-3],
                                                                       Gann_list[insrt_loc-4], Gann_list[insrt_loc-5])

  buy_above = Gann_list[insrt_loc+1]
  c1 = 0.99995
  sell_target_1, sell_target_2, sell_target_3, sell_target_4 =  (round(c1*Resistance_1,2), round(c1*Resistance_2,2), 
                                                                              round(1*Resistance_3,2), round(c1*Resistance_4,2))
  sl_long = Gann_list[insrt_loc-1]
  sell_below = Gann_list[insrt_loc-1]
  c2 = 1.00005
  buy_target_1, buy_target_2, buy_target_3, buy_target_4 = (round(c2*Support_1,2), round(c2*Support_2,2), round(c2*Support_3,2), 
                                                                        round(c2*Support_4,2))
  sl_short = Gann_list[insrt_loc+1]
  
  return (buy_above, sl_long, sell_target_1, sell_target_2, sell_target_3, sell_target_4,
          sell_below, sl_short, buy_target_1, buy_target_2, buy_target_3, buy_target_4)
  

In [5]:
def profit_percentage(prediction_list_df): # this function will use multiple levls of filtering
    
    stock_tckr = prediction_list_df['stock_tckr']
    buy_price =  prediction_list_df['buy_above'] #buying price for long trade
    sell_price = prediction_list_df['sell_below'] # selling price for short trade
    sl_long = prediction_list_df['sl_long']
    sl_short = prediction_list_df['sl_short']

    # Percentage profit with target 1 achieved for long trade
    pct_profit_long_tgt_1 = (prediction_list_df['sell_target_1'] - buy_price) * 100/ buy_price
    # Percentage profit with target 2 achieved for long trade
    pct_profit_long_tgt_2 = (prediction_list_df['sell_target_2'] - buy_price) * 100/ buy_price
    # Percentage profit with target 3 achieved for long trade
    pct_profit_long_tgt_3 = (prediction_list_df['sell_target_3'] - buy_price) * 100/ buy_price
    # Percentage profit with target 4 achieved for long trade
    pct_profit_long_tgt_4 = (prediction_list_df['sell_target_4'] - buy_price) * 100/ buy_price

    # Percentage profit with target 1 achieved for short trade
    pct_profit_short_tgt_1 = (sell_price - prediction_list_df['buy_target_1']) *100/ prediction_list_df['buy_target_1']
    # Percentage profit with target 2 achieved for short trade
    pct_profit_short_tgt_2 = (sell_price - prediction_list_df['buy_target_2']) *100/ prediction_list_df['buy_target_2']
    # Percentage profit with target 3 achieved for short trade
    pct_profit_short_tgt_3 = (sell_price - prediction_list_df['buy_target_3']) *100/ prediction_list_df['buy_target_3']
    # Percentage profit with target 4 achieved for short trade
    pct_profit_short_tgt_4 = (sell_price - prediction_list_df['buy_target_4']) *100/ prediction_list_df['buy_target_4']
    
    # check stop loss vs pct profit
    # Risk reward percentage: long trade vs stop loss
    loss_pct_against_long_trade = (buy_price - sl_long) * 100/ buy_price
    # Risk reward percentage: short trade vs stop loss
    loss_pct_against_short_trade = (sl_short - sell_price) * 100/ sl_short

    return (stock_tckr, pct_profit_long_tgt_1, pct_profit_long_tgt_2, pct_profit_long_tgt_3, pct_profit_long_tgt_4,
            pct_profit_short_tgt_1, pct_profit_short_tgt_2, pct_profit_short_tgt_3, pct_profit_short_tgt_4,
            loss_pct_against_long_trade, loss_pct_against_short_trade)

In [6]:
def stock_filtering(long_profit, long_loss, short_profit, short_loss):
    if((long_profit>(2*long_loss) or short_profit>(2*short_loss)) and (long_profit>2 or short_profit>2)):
        return "good_stock"
    else:
        return "remove_stock"


In [7]:
# Main 
intraday_stock_list = get_intraday_stocklist()
prediction_list =[]
risk_rewards = []
#current_date = date.today().strftime("%Y-%m-%d")
start_date = pd.Timestamp('today') - timedelta(days=1)# previous 1 day from today
start_date = start_date.strftime("%Y-%m-%d")

current_date = pd.Timestamp('today') + timedelta(days=1) # this adjustment is required because python does not include last date of the range.
current_date = current_date.strftime("%Y-%m-%d")


In [8]:
# Gann prediction algorithm
for i in range(len(intraday_stock_list)-1):
#for i in range(0,10): # test with 3 stocks
  intraday_stock_tckr = intraday_stock_list['Intraday'][i] # get ticker symbol
  intraday_stock_data = download_stock(intraday_stock_tckr, start_date ,current_date) # download data from yfinance 
  if(len(intraday_stock_data)>0 and not(pd.isnull(intraday_stock_data.iloc[0,3]))):
    current_day_close = round(intraday_stock_data.values[-1][3],2)

    (buy_above, sl_long, sell_target_1, sell_target_2, sell_target_3, sell_target_4,
           sell_below, sl_short, buy_target_1, buy_target_2, buy_target_3, buy_target_4)= Gann_prediction(current_day_close)
          
    prediction_list.append([intraday_stock_tckr,buy_above, sl_long, sell_target_1, sell_target_2, sell_target_3, sell_target_4,
           sell_below, sl_short, buy_target_1, buy_target_2, buy_target_3, buy_target_4])

prediction_list_df = pd.DataFrame(prediction_list, columns=['stock_tckr','buy_above', 'sl_long', 'sell_target_1', 'sell_target_2', 
        'sell_target_3', 'sell_target_4', 'sell_below', 'sl_short', 'buy_target_1', 'buy_target_2', 'buy_target_3', 'buy_target_4'])



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:
risk_rewards =[]

In [ ]:
#filtering low profit stocks
#for i in range(0,100): # test with 3 stocks
for i in range(len(prediction_list_df)):
  (stock_tckr, pct_profit_long_tgt_1, pct_profit_long_tgt_2, pct_profit_long_tgt_3, pct_profit_long_tgt_4,
  pct_profit_short_tgt_1, pct_profit_short_tgt_2, pct_profit_short_tgt_3, pct_profit_short_tgt_4,
  loss_pct_against_long_trade, loss_pct_against_short_trade) = profit_percentage(prediction_list_df.iloc[i])

  risk_rewards.append([stock_tckr, pct_profit_long_tgt_1, pct_profit_long_tgt_2, pct_profit_long_tgt_3, pct_profit_long_tgt_4,
  pct_profit_short_tgt_1, pct_profit_short_tgt_2, pct_profit_short_tgt_3, pct_profit_short_tgt_4,
  loss_pct_against_long_trade, loss_pct_against_short_trade])

risk_rewards_df = pd.DataFrame(risk_rewards, columns = ['stock_tckr', 'pct_profit_long_tgt_1', 'pct_profit_long_tgt_2', 'pct_profit_long_tgt_3',
 'pct_profit_long_tgt_4', 'pct_profit_short_tgt_1', 'pct_profit_short_tgt_2', 'pct_profit_short_tgt_3', 'pct_profit_short_tgt_4', 
 'loss_pct_against_long_trade', 'loss_pct_against_short_trade'])

# Merge both data frames, Gann predictions and risk rewards for evaluation
reduced_stocks_df = pd.merge(prediction_list_df, risk_rewards_df,  on = 'stock_tckr')

# Filter elimination criteria
reduced_stocks_df['stock_filter'] = reduced_stocks_df.apply(lambda x: stock_filtering(x.pct_profit_long_tgt_2, x.loss_pct_against_long_trade, x.pct_profit_short_tgt_2,
                                                        x.loss_pct_against_short_trade), axis=1)
reduced_stocks_df = reduced_stocks_df[reduced_stocks_df['stock_filter']!='remove_stock'] # drop low profit stocks
reduced_stocks_df=reduced_stocks_df.reset_index(drop=True) #reset index


EMA strategy to get direction

In [ ]:
start_date = pd.Timestamp('today') - timedelta(days=30) # previous 1 day from today
start_date = start_date.strftime("%Y-%m-%d")


In [ ]:
# Intraday trades
signal_list =[]
for i in range(len(reduced_stocks_df)):
  #for i in range(0,10):
  intraday_stock_tckr = reduced_stocks_df['stock_tckr'][i] # get ticker symbol
  data = download_stock(intraday_stock_tckr, start_date ,current_date) # download data from yfinance 
  if(len(data)>0):
    #calculate EMA and decide the direction
    ema = ta.EMA(data.Close, timeperiod=13)
    
    #long_ema = ta.EMA(data['Close'], timeperiod=50)
    #first 15 min candle of the day
    previous_high = data.High[-2]
    latest_high = data.High[-1]
    previous_low = data.Low[-2]
    latest_low = data.Low[-1]
    previous_close = data.Close[-2]
    latest_close = data.Close[-1]


    if(latest_close<ema[-1]):
      signal ='sell'
    if((previous_close < ema[-1]) and (latest_close<previous_low)):
      signal = 'buy'
    else:
      signal = 'none'
    
    signal_list.append([intraday_stock_tckr, signal])




#create df of signal direction
signal_list_df = pd.DataFrame(signal_list, columns =['stock_tckr', 'signal'])

In [ ]:
# create a df to backtest using actual data against predicted data
final_stocks_for_tomorrow = pd.merge(reduced_stocks_df,signal_list_df, on = 'stock_tckr')

In [ ]:
final_stocks_for_tomorrow.rename(columns={'stock_tckr': 'Stock symbol', 'buy_above': 'Buy above', 
'sl_long': 'Stop loss long', 'sell_target_1':'Sell target 1','sell_target_2':'Sell target 2','sell_target_3':'Sell target 3',
'sell_target_4':'Sell target 4', 'sell_below':'Sell below', 'sl_short':'Stop loss short', 'buy_target_1':'Buy target 1', 'buy_target_2':'Buy target 2',
'buy_target_3':'Buy target 3', 'buy_target_4':'Buy target 4',
'pct_profit_long_tgt_4':'Max Gain(%) long', 'pct_profit_short_tgt_4':'Max Gain(%) short',
'signal':'Action'}, inplace=True)

In [ ]:
final_stocks_for_tomorrow.drop(['pct_profit_long_tgt_1', 'pct_profit_long_tgt_2', 'pct_profit_long_tgt_3','pct_profit_short_tgt_1', 
'pct_profit_short_tgt_2','pct_profit_short_tgt_3', 'loss_pct_against_long_trade', 'loss_pct_against_short_trade','stock_filter'], 
       axis=1, inplace=True)

In [ ]:
tomorrow=date.today() + timedelta(days=1)
final_stocks_for_tomorrow.insert(loc = 0,
          column = 'Date',
          value = tomorrow)
final_stocks_for_tomorrow = final_stocks_for_tomorrow[final_stocks_for_tomorrow.Action != 'none']

In [ ]:
final_stocks_for_tomorrow.to_csv('Predictions/intraday_predictions_for_tomorrow.csv')